In [1]:
from sagemaker.huggingface import HuggingFaceModel
from sagemaker.async_inference.async_inference_config import AsyncInferenceConfig
import sagemaker
import boto3

In [2]:
role = sagemaker.get_execution_role()

In [3]:
# Hub Model configuration. https://huggingface.co/models
hub = {
    'HF_MODEL_ID':'mrm8488/distilroberta-finetuned-age_news-classification',
    'HF_TASK':'text-classification'
}

In [4]:
# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
    transformers_version='4.17.0',
    pytorch_version='1.10.2',
    py_version='py38',
    env=hub,
    role=role,
)

In [5]:
# async inference config
async_config = AsyncInferenceConfig(
    output_path='s3://daily-digest-v1-classification-results',
    max_concurrent_invocations_per_instance=4
)

In [6]:
# deploy model to SageMaker Inference
endpoint_name='huggingface-classification-inference'

predictor = huggingface_model.deploy(
    initial_instance_count=1, # number of instances
    instance_type='ml.m5.xlarge', # ec2 instance type
    async_inference_config=async_config,
    endpoint_name=endpoint_name
)

-----!

In [8]:
predictor.predict({
    'inputs': "The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building, and the tallest structure in Paris. Its base is square, measuring 125 metres (410 ft) on each side. During its construction, the Eiffel Tower surpassed the Washington Monument to become the tallest man-made structure in the world, a title it held for 41 years until the Chrysler Building in New York City was finished in 1930. It was the first structure to reach a height of 300 metres. Due to the addition of a broadcasting aerial at the top of the tower in 1957, it is now taller than the Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, the Eiffel Tower is the second tallest free-standing structure in France after the Millau Viaduct."
})

[{'label': 'Sci/Tech', 'score': 0.9641291499137878}]

In [16]:
# test async endpoint

smr_client = boto3.client('sagemaker-runtime')
response = smr_client.invoke_endpoint_async(
    EndpointName=endpoint_name,
    InputLocation='s3://daily-digest-v1-inference-payload/sourcehash1234',
    ContentType='application/json'
)

print(response)

{'ResponseMetadata': {'RequestId': 'aaed9d99-ff4f-4902-9bf6-340eeec93174', 'HTTPStatusCode': 202, 'HTTPHeaders': {'x-amzn-requestid': 'aaed9d99-ff4f-4902-9bf6-340eeec93174', 'x-amzn-sagemaker-outputlocation': 's3://daily-digest-v1-classification-results/30d73c20-740c-4587-aa8a-7fd7c652b629.out', 'date': 'Tue, 29 Nov 2022 02:28:35 GMT', 'content-type': 'application/json', 'content-length': '54'}, 'RetryAttempts': 0}, 'OutputLocation': 's3://daily-digest-v1-classification-results/30d73c20-740c-4587-aa8a-7fd7c652b629.out', 'InferenceId': 'ba6bae7b-6d87-4ca5-8026-f30d91a69f57'}
